In [92]:
import os
from langchain_community.document_loaders import PyPDFLoader

data_path = '../../data/UNVXIS.05 Enable Internal Sales Orders.pdf'

loader = PyPDFLoader(data_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)
pages

[Document(metadata={'source': '../../data/UNVXIS.05 Enable Internal Sales Orders.pdf', 'page': 0}, page_content=' \n \n \n \n \n \n \n \n \n \n \n  \n  \n \nEnable Internal \nSales Orders \nUNVXIS.05 \nApplication: Infor M3 \nProcess: Internal Sales Orders \n \n \n'),
 Document(metadata={'source': '../../data/UNVXIS.05 Enable Internal Sales Orders.pdf', 'page': 1}, page_content='Enable Internal Sales Orders (Internal Sales Orders) 2 (13) \nTable of contents \nOVERVIEW ............................................................................................................................... 3 \nCONCEPT ................................................................................................................................. 3 \nPROCESS ................................................................................................................................. 3 \nINPUT .........................................................................................................

In [144]:
document_text = "\n".join([page.page_content for page in pages[2:]])
document_text = document_text[70:]
document_text

" \nConcept \n \nProcess  \nThis configuration process explains how the system must be set up to enable internal sales \norders. One major consideration is the Goods-in-transit (GIT) management. As explained in \nthe detailed steps, this is controlled by a parameter in ‘Settings – Cost Accounting’ \n(CAS900), which is set Off. \nThe project team should decide whether to activate this, as it will have an impact on internal \nsales orders accounting and “external”, or “regular” customer orders and purchase orders. \nAlthough the Goods-in-transit functionality has not been activated, the process descriptions \nassume that it has been. \nInput  \n \nOutput  \nThe settings and basic data are in place for using internal sales orders. \nThe project team has chosen whether to activate the Goods-in-transit (GIT) functionality. \nDepending on this, GIT will be managed or not during the internal sales orders process. \n  \nEnable Internal Sales Orders (Internal Sales Orders) 4 (13) \nTasks  \nMan

In [ ]:
import re

# Define the headers based on the table of contents
headers = [
    "Overview", "Concept", "Process", "Input", "Output", "Tasks",
    "Manage General Settings", "Manage PO Settings", "Manage Internal CO Settings",
    "Manage Item Settings", "Manage Goods-in-Transit Settings", "Setup",
    "Additional Information", "Test Variants", "Tips and Hints", "Kits",
    "Attributes", "QMS (Quality Management System)", "Historic Actual Cost (HAC) and Goods-in-transit (GIT)",
    "Line Charges", "Supply Chain Orders", "Transportation lead time"
]

def extract_headers_and_content(text, headers):
    # Create a dictionary to store headers and their content
    header_dict = {}
    
    # Iterate over the headers and extract their content
    for i, header in enumerate(headers):
        pattern = re.compile(rf'^\s*{re.escape(header)}\s*$', re.MULTILINE)
        match = pattern.search(text)
        if match:
            start = match.end()
            next_match = pattern.search(text, start)
            end = next_match.start() if next_match else len(text)
            content = text[start:end].strip()
            header_dict[header] = content
    
    return header_dict

content = extract_headers_and_content(document_text, headers)
content

AttributeError: 'NoneType' object has no attribute 'start'

In [146]:
import re

def extract_sections(text):
    # Define the pattern to match the headers
    header_pattern = re.compile(r'\n(?P<header>Concept|Process|Input|Output)\s*\n')
    
    # Find all headers
    headers = list(header_pattern.finditer(text))
    
    # Create a dictionary to store headers and their content
    header_dict = {}
    
    # Iterate over the headers and store them in the dictionary
    for i, match in enumerate(headers):
        header = match.group('header').strip()
        start = match.end()
        end = headers[i + 1].start() if i + 1 < len(headers) else len(text)
        content = text[start:end].strip()
        header_dict[header] = content if content else None
    
    return header_dict

# Extract sections from the document text
sections = extract_sections(document_text)
sections

{'Concept': 'Process  \nThis configuration process explains how the system must be set up to enable internal sales \norders. One major consideration is the Goods-in-transit (GIT) management. As explained in \nthe detailed steps, this is controlled by a parameter in ‘Settings – Cost Accounting’ \n(CAS900), which is set Off. \nThe project team should decide whether to activate this, as it will have an impact on internal \nsales orders accounting and “external”, or “regular” customer orders and purchase orders. \nAlthough the Goods-in-transit functionality has not been activated, the process descriptions \nassume that it has been.',
 'Input': "Output  \nThe settings and basic data are in place for using internal sales orders. \nThe project team has chosen whether to activate the Goods-in-transit (GIT) functionality. \nDepending on this, GIT will be managed or not during the internal sales orders process. \n  \nEnable Internal Sales Orders (Internal Sales Orders) 4 (13) \nTasks  \nManage G

In [73]:
def save_sections(section_dict):
    for header, content in section_dict.items():
        # Create a filename based on the header
        filename = f"{header.replace(' ', '_').lower()}.txt"
        os.removedirs('sections')
        os.makedirs('sections', exist_ok=True)
        filepath = os.path.join('sections', filename)
        # Save the content to a file
        with open(filepath, 'w') as file:
            file.write(content)

# Save each section to a separate file
save_sections(sections)

OSError: [Errno 39] Directory not empty: 'sections'

In [ ]:
for header, content in sections.items():
    print(header)
    print(content)

OVERVIEW

CONCEPT

PROCESS

INPUT

OUTPUT

TASKS

MANAGE GENERAL SETTINGS
1. Activate Internal Sales Orders ....................................................................................................................... 4 
2. Configure Stock Messages................................................................................................................................ 4 
3. Manage Deviations ............................................................................................................................................. 4
MANAGE PO SETTINGS
1. Define PO Type ..................................................................................................................................................... 5 
2. Define Suppliers ................................................................................................................................................... 5 
3. Define PO Charges ...............................................................